<a href="https://colab.research.google.com/github/anumadhyani/mywork/blob/master/Detect_AI_images_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0, MobileNetV2, ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Concatenate
import numpy as np

class DeepFeatureExtractor:
    """
    Extract deep features using multiple pre-trained CNNs
    Combines EfficientNet, MobileNet, and ResNet for robust representation
    """

    def __init__(self):
        # Load multiple pre-trained models
        self.efficientnet = EfficientNetB0(weights='imagenet', include_top=False, pooling='avg')
        self.mobilenet = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')
        self.resnet = ResNet50V2(weights='imagenet', include_top=False, pooling='avg')

        # Freeze pre-trained layers
        for model in [self.efficientnet, self.mobilenet, self.resnet]:
            model.trainable = False

    def extract_features(self, image):
        """
        Extract features from image using all three networks

        Returns:
            Combined feature vector (1280 + 1280 + 2048 = 4608 dimensions)
        """
        # Preprocess for each network
        img_resized = cv2.resize(image, (224, 224))
        img_array = np.expand_dims(img_resized, axis=0)

        # EfficientNet preprocessing
        from tensorflow.keras.applications.efficientnet import preprocess_input as eff_preprocess
        eff_input = eff_preprocess(img_array.copy())
        eff_features = self.efficientnet.predict(eff_input, verbose=0)

        # MobileNet preprocessing
        from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mob_preprocess
        mob_input = mob_preprocess(img_array.copy())
        mob_features = self.mobilenet.predict(mob_input, verbose=0)

        # ResNet preprocessing
        from tensorflow.keras.applications.resnet_v2 import preprocess_input as res_preprocess
        res_input = res_preprocess(img_array.copy())
        res_features = self.resnet.predict(res_input, verbose=0)

        # Concatenate all features
        combined_features = np.concatenate([
            eff_features.flatten(),
            mob_features.flatten(),
            res_features.flatten()
        ])

        return combined_features


class DeepAIDetector:
    """
    End-to-end deep learning detector with fine-tuning
    """

    def __init__(self, feature_extractor):
        self.feature_extractor = feature_extractor
        self.classifier = None

    def build_classifier(self, input_dim=4608):
        """
        Build classification head on top of deep features
        """
        from tensorflow.keras import Sequential
        from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
        from tensorflow.keras.regularizers import l2

        self.classifier = Sequential([
            Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
            BatchNormalization(),
            Dropout(0.5),

            Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
            BatchNormalization(),
            Dropout(0.4),

            Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
            BatchNormalization(),
            Dropout(0.3),

            Dense(64, activation='relu'),
            Dropout(0.2),

            Dense(1, activation='sigmoid')  # Binary classification
        ])

        self.classifier.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss='binary_crossentropy',
            metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
        )

    def train(self, X_train, y_train, X_val, y_val, epochs=50):
        """
        Train the classifier
        """
        from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

        callbacks = [
            EarlyStopping(monitor='val_auc', patience=10, restore_best_weights=True, mode='max'),
            ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-7),
            ModelCheckpoint('best_model.h5', monitor='val_auc', save_best_only=True, mode='max')
        ]

        history = self.classifier.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=32,
            callbacks=callbacks,
            verbose=1
        )

        return history

    def predict(self, features):
        """
        Predict with confidence scores
        """
        prediction = self.classifier.predict(features.reshape(1, -1), verbose=0)[0][0]

        return {
            'prediction': 1 if prediction > 0.5 else 0,
            'confidence': prediction if prediction > 0.5 else 1 - prediction,
            'probability_ai': float(prediction),
            'probability_real': float(1 - prediction)
        }